# Introdução

### Tema:

Consumo de alcool por estudantes.

### Pergunta

Dado o sexo, a idade, a educação formal da mãe e do pai, o tempo de estudo semanal, a qualidade da relação familiar e o tempo livre depois da escola, quão provável o estudante estudante irá consumir alcool durante dias úteis e durante finais de semana?

### Objetivo:

Prever um rótulo (target qualitativo), variando entre 5 escalas de "pouco provável" até "muito provável".

### Técnica Utilizada

Classificação

*_Colocar aqui a justificativa para a técnica_*

### Base de Dados

*_Colocar aqui a descrição das colunas analisadas igual no Kaggle_*

### Trabalhos Acadêmicos

*_Colocar aqui publicações que relacionem dados socioeconômicos com consumo de alcool por estudantes_*

### Bibliotecas utilizadas

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import csv
import matplotlib.pyplot as plt

# Minerando Dados e Características do Dataset

In [2]:
student_mat_data = pd.read_csv('student-mat.csv')

student_por_data = pd.read_csv('student-por.csv')

base = pd.concat([student_por_data,student_mat_data],ignore_index=True)

base

### Necessário juntar em uma única base de dados, retirar colunas não utilizadas, linhas duplicadas e linhas vazias

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
1040,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
1041,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
1042,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [3]:
base.drop(['G1', 'G2'], axis=1)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,4,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,2,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,6,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,0,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039,MS,M,20,U,LE3,A,2,2,services,services,...,no,no,5,5,4,4,5,4,11,9
1040,MS,M,17,U,LE3,T,3,1,services,services,...,yes,no,2,4,5,3,4,2,3,16
1041,MS,M,21,R,GT3,T,1,1,other,other,...,no,no,5,5,3,3,3,3,3,7
1042,MS,M,18,R,LE3,T,3,2,services,other,...,yes,no,4,4,1,3,4,5,0,10


In [4]:
base.isnull().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64

In [16]:
falsas_numericas = ['Medu','Fedu','famrel','freetime','goout','freetime','Dalc','Walc','health',]
categoricas = base.select_dtypes(exclude=[np.number]).columns

for column in falsas_numericas:
    base[column] = base[column].astype('category')

for column in categoricas:
    base[column] = base[column].astype('object')
    
num_base = base.select_dtypes(include=[np.number]).columns
cat_base = base.select_dtypes(exclude=[np.number]).columns
print(num_base)
print("\n {}".format(cat_base))

Index(['age', 'traveltime', 'studytime', 'failures', 'absences', 'G1', 'G2',
       'G3'],
      dtype='object')

 Index(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid',
       'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health'],
      dtype='object')


# Modelos de Predição

# Processo e Estatísticas de Validação

# Conclusão

# Referências Bibliográficas